In [2]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
spark = SparkSession(sc)
sqlContext=SQLContext(sc)

In [3]:
temps= sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/temperature-readings-tiny.csv")
parts = temps.map(lambda l:l.split(";"))
tempReadings = parts.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0],month=p[1].split("-")[1],day=p[1].split("-")[2], time=p[2],  Temp=float(p[3]), quality=p[4]))
#Inferring the schema and registering the DataFrame as atable
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")


schemaAvg=schemaTempReadings.select('year','month','day','station','Temp')\
.filter((schemaTempReadings['year'] <=2014) & (schemaTempReadings["year"]>=1960 )).groupby( "year","month",'day','station')\
.agg( (F.avg('Temp') ).alias("daily_avg_temp"))\
.select('year','month','station','daily_avg_temp').groupBy('year','month','station')\
.agg( (F.avg('daily_avg_temp') ).alias("avg_temperature"))\
.orderBy('year','month','station',ascending=False)

schemaAvg.show()


+----+-----+-------+--------------------+
|year|month|station|     avg_temperature|
+----+-----+-------+--------------------+
|2014|   12| 102170|  -5.801612903225807|
|2014|   11| 102170|               2.525|
|2014|   10| 102170|   7.106451612903227|
|2014|   09| 102170|   8.584999999999999|
|2014|   08| 102170|  13.869354838709677|
|2014|   07| 102170|  19.659677419354836|
|2014|   06| 102170|  14.443333333333332|
|2014|   05| 102170|  10.756451612903223|
|2014|   04| 102170|   4.776666666666666|
|2014|   03| 102170|   1.896774193548387|
|2014|   02| 102170|  0.6749999999999999|
|2014|   01| 102170|  -4.106451612903227|
|2013|   12| 102170|  0.7096774193548387|
|2013|   11| 102170|-0.05166666666666...|
|1961|   05| 102190|   10.61533333333333|
|1961|   04| 102190|   4.507777777777779|
|1961|   03| 102190|   2.708602150537634|
|1961|   02| 102190| -2.1333333333333337|
|1961|   01| 102190|  -7.215053763440859|
|1960|   12| 102190| -2.4274193548387086|
+----+-----+-------+--------------